# Goals
The aim is to recreate FE versions of the test problems form the BICGStab paper and export them as mtx files.

## Set Up
This is assembled from the Gridap tutorials.  It is set up to be as understandable as possible.  This just so that I can see where everything came from. 

In [30]:
using Gridap, LinearAlgebra, LimitedLDLFactorizations, Preconditioners, SparseArrays

n = 34; Omega = (0.0,pi,0.0,pi);
Grid = CartesianDiscreteModel(Omega, (n,n))
# Grid has 4 corners and 4 edges
# "boundary" is all elements of the boundary of Omega
# Tags 1-4 are the corners and Tags 5-8 are the edges.
# Label each of the six sides differently
# boundary
Tags = get_face_labeling(Grid)
add_tag_from_tags!(Tags,"y=0",[1,2,5])
add_tag_from_tags!(Tags,"OtherEdges",[3,4,6,7,8])
# Piecewise linear Lagrange elements
order = 1; Elements = ReferenceFE(lagrangian,Float64,order)
V = TestFESpace(Grid,Elements;conformity=:H1,dirichlet_tags="boundary")
# Incorporating Homogeneous Dirichlet boundary conditions.
g(x) = 0.0
U = TrialFESpace(V,g)
# Triangulating and defining measures
degree = 2
Ω = Triangulation(Grid)
dΩ = Measure(Ω,degree)
# Setting up a simple test problem
f(x) = 1.0
a(u,v) = ∫( ∇(v)⋅∇(u) )*dΩ
b(v) = ∫( v*f )*dΩ
# Set up FE operator
op = AffineFEOperator(a,b,U,V)
# Extract Stiffness Matrix A
A=get_matrix(op)
# Compute Incomplete Cholesky Decomposition
# Dimensions needed to inhibit pivoting by specifying "natural order" 1:m 
# print(fieldnames(LimitedLDLFactorization))
m=size(A)[1]
LStructure = lldl(tril(A,-1), diag(A), 1:m, memory=1)
L=LStructure.L+I
D=spdiagm(LStructure.D)
# alpha should be zero and p should be the identity permutation
# \alpha then TAB produces pretty α
alpha=LStructure.α
p=LStructure.P
println("α=",alpha," p==1:m is ",p==1:m) 
# Check preconditioner quality
println("relative residual error L form is ", norm(A - L*D*L')/norm(A))
P=L*spdiagm(sqrt.(abs.(LStructure.D)));
println("relative residual error P form is ", norm(A - P*P')/norm(A))

α=0.0 p==1:m is true
relative residual error L form is 0.0335817330731796
relative residual error P form is 0.0335817330731796


# Problem 1
$-\Delta u$
All Homogeneous Dirichlet conditions

# Problem 2
$-\nabla \cdot (D(x,y) *\nabla u)$
Homogeneous Dirichlet conditions only on y=0

# Problem 3
$-\nabla \cdot (D(x,y) *\nabla u)$
Homogeneous Dirichlet conditions only on y=0

# Problem 4
$-\nabla \cdot (D(x,y) *\nabla u)$
Homogeneous Dirichlet conditions only on y=0